In [1]:
from PIL import Image
from multiprocessing.dummy import Pool as ThreadPool
import multiprocessing
import numpy as np
import cv2
import os
import sys
import io


In [6]:
# define the path to the face detector
FACE_DETECTOR_PATH = "/home/gaurav/Documents/GAURAV/Test/AML/cut_api-master/cascades/haarcascade_frontalface_default.xml"
OUT_PATH = "/home/gaurav/Documents/GAURAV/Test/AML/cut_api-master/out/"
# format(base_path=os.path.abspath(os.path.dirname(__file__)))
# DOWNLOAD_PATH = "{base_path}/images".format(base_path=os.path.abspath(os.path.dirname(__file__)))

In [32]:
def facedetection(image):
    # convert the image to grayscale, load the face cascade detector and detect faces in the image
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    detector = cv2.CascadeClassifier(FACE_DETECTOR_PATH)
    drects = detector.detectMultiScale(img_gray, scaleFactor=1.1, minNeighbors=5,
                                      minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

    # construct a list of bounding boxes from the detection
    drects = [(int(x), int(y), int(x + w), int(y + h)) for (x, y, w, h) in drects]
    
    # Ignore false faces since multiple faces are detected on certain images
    height,width = img_gray.shape
    drects.sort(key=lambda x:x[1],reverse=False)
    if len(drects)>0:
       rects = drects[:1]
       lx,ly,rx,ry = rects[0]
       if ry>height/2 or np.var( img_gray[ly:ry,lx:rx])<1:
          rects = [] 
    else:
       rects = []
 
    return rects

def findbound(img):
    height,width = img.shape

    # Find the rectangle boundary in four directions
    for i in range(width):
        if img[:,i].tolist().count(255) > 2:
           left = i
           break

    for i in range(width-1,-1,-1):
        if img[:,i].tolist().count(255) > 2:
           right = i
           break

    for i in range(height):
        if img[i,:].tolist().count(255) > 2:
           top = i
           break

    for i in range(height-1,-1,-1):
        if img[i,:].tolist().count(255) > 2:
           bottom = i
           break

    bound = [(top,bottom,left,right)]
    return bound

def filterboundarea(img, bvalue, svalue, rect):
    top,bottom,left,right = rect

    mask = np.zeros(img.shape[:2],np.uint8)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     img_gray =img

    for (y,x), value in np.ndenumerate(img_gray):
        if y>top and y<bottom and x>left and x<right:
           if img_gray[y,x] > bvalue+20 or img_gray[y,x]<bvalue-20:
              mask[y,x] = 1

    if not np.isnan(svalue):
       for (y,x), value in np.ndenumerate(img_gray):
        if y>top and y<bottom and x>left and x<right:
           if img_gray[y,x] < svalue+50 and img_gray[y,x]>svalue-50:
              mask[y,x] = 0

    fimg = img*mask[:,:,np.newaxis]

    return fimg

def backgroundmodel(img,rect):
    top,bottom,left,right = rect
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    mask = np.ones(img.shape[:2],np.uint8)
    mask[top:bottom,left:right]=0
    img_gray = img_gray*mask
    bvalue = np.median(img_gray[img_gray>0])

    return bvalue

def skinmodel(img,rect):
    startX,startY,endX,endY = rect
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    crop_img = img_gray[startY:endY, startX:endX]
    
    hist = [0]*256 
    for (y,x),value in np.ndenumerate(crop_img):
        hist[crop_img[y,x]] += 1 

    return hist.index(max(hist))



def removebackground(image,rects):
    # select left corner 15x15 patch
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_blur = cv2.GaussianBlur(img_gray, (3, 3), 0)
    height,width = img_gray.shape

    v = np.average(img_gray[1:16])
    sigma = 0.33
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(img_blur, lower, upper)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    dilated = cv2.dilate(edged, kernel, iterations = 3)
    
    # Find the bound area
    boundrect = findbound(dilated)

    # Get the background RGB model
    bvalue = backgroundmodel(image, boundrect[0])
    
    #Get the skin model
    if len(rects)>0:
       svalue = skinmodel(image,rects[0])
    else:
       svalue = np.nan

    # Exclude face area
    if len(rects)>0:
        lx,ly,rx,ry = rects[0]
        top,bottom,left,right = boundrect[0]
        boundrect=[]
        boundrect = [(ry,bottom,left,right)]

    # Filter out background in bounded area
    rgimg = filterboundarea(image, bvalue, svalue, boundrect[0])
    return rgimg 

def extractobj(filepath,OUT_PATH):
    image = cv2.imread(filepath)
    filename = filepath.split('/')[-1] 
    dotidx = filename.index('.') 
#     new_filename = OUT_PATH + filename[:dotidx] + "_cut" + filename[dotidx:]
    
#     print(new_filename)
 
    # face detection 
    rects = facedetection(image)

    # detect and remove background
    fimg = removebackground(image,rects)
#     fimg = cv2.cvtColor(fimg,cv2.COLOR_GRAY2RGB)
 
#     cv2.imwrite(new_filename,fimg)
    cv2.imwrite(OUT_PATH,fimg)


In [53]:
def create_filter_folder(input_path):
#     file_list = list(os.walk(input_path))
#     folders = file_list[1]
    folders = os.listdir(input_path)
    for folder in folders:
        print('Processing Folder %s'%folder)
        if os.path.exists(input_path+'/'+folder+'_filter')==True:
            print('folder_existed %s'%input_path+'/'+folder+'_filter')
        else:
            os.makedirs(input_path+'/'+folder+'_filter')
            print('folder made : %s'%input_path+'/'+folder+'_filter')
        out_path =  str(input_path)+'/'+str(folder)+'_filter'
        
        in_path = str(input_path)+'/'+str(folder)
        
        list_folder = os.listdir(str(in_path))
        for fld in list_folder:
            list_files = os.listdir(str(in_path)+'/'+str(fld))
            for file in list_files:
                in_ = str(in_path)+'/'+str(fld)+'/'+str(file)
            
                out_path_ = out_path+'/'+str(fld)+'/'+str(file)
                
                if os.path.exists(out_path+'/'+str(fld))==True:
                    pass
                else:
                    os.makedirs(out_path+'/'+str(fld))
                    print('folder_made : %s'%out_path+'/'+str(fld))
                try:
                    extractobj(in_,out_path_)
                except Exception as e:
                    print(e)
                    continue
#                 crop_image(in_,out_path_)
                
    return 'DONE'

In [55]:
create_cropped_folder(input_path='/home/gaurav/Documents/GAURAV/Test/AML/data_orginal/opencv')

Processing Folder validation
folder made : /home/gaurav/Documents/GAURAV/Test/AML/data_orginal/opencv/validation_crop
folder_made : /home/gaurav/Documents/GAURAV/Test/AML/data_orginal/opencv/validation_crop/typography
folder_made : /home/gaurav/Documents/GAURAV/Test/AML/data_orginal/opencv/validation_crop/solid
folder_made : /home/gaurav/Documents/GAURAV/Test/AML/data_orginal/opencv/validation_crop/striped
folder_made : /home/gaurav/Documents/GAURAV/Test/AML/data_orginal/opencv/validation_crop/graphic
folder_made : /home/gaurav/Documents/GAURAV/Test/AML/data_orginal/opencv/validation_crop/abstract
folder_made : /home/gaurav/Documents/GAURAV/Test/AML/data_orginal/opencv/validation_crop/Colourblock
folder_made : /home/gaurav/Documents/GAURAV/Test/AML/data_orginal/opencv/validation_crop/Patterned
folder_made : /home/gaurav/Documents/GAURAV/Test/AML/data_orginal/opencv/validation_crop/Melange
folder_made : /home/gaurav/Documents/GAURAV/Test/AML/data_orginal/opencv/validation_crop/polka dot

'DONE'

In [50]:
files = os.listdir('/home/gaurav/Documents/GAURAV/Test/AML/data_orginal/opencv/validation/Patterned/')

In [51]:
for i in files:
    in_ = '/home/gaurav/Documents/GAURAV/Test/AML/data_orginal/opencv/validation/Patterned/'+i
    out_ = '/home/gaurav/Documents/GAURAV/Test/AML/data_orginal/opencv/validation_crop/Patterned/'+i
    print(i)
    extractobj(in_,out_)

    
    

67011.jpg
66936.jpg
67019.jpg
66994.jpg
67085.jpg
66909.jpg
67077.jpg
67047.jpg
66862.jpg
67127.jpg
66940.jpg
66944.jpg
67075.jpg
66977.jpg
67039.jpg
67049.jpg
66877.jpg
67022.jpg
67079.jpg
66980.jpg
66902.jpg
66960.jpg
66868.jpg
66938.jpg
66914.jpg
66907.jpg
67103.jpg
67083.jpg
